In [ ]:
import os
import re
import nltk
import openpyxl # type: ignore
from nltk.stem import PorterStemmer

In [ ]:
DATA_URL  = '/Users/mohamad/Documents/GitHub/BrainQuest/data/documents/Cranfield collection/'

Create Stemmer


In [ ]:
stemmer = PorterStemmer()

In [ ]:
def load_stopwords(file_path):
    try:
        with open(file_path, "r") as stop_file:
            content = stop_file.read()
            stopwords = content.split(",")
            return stopwords
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return []

stopwords_file_path = "/Users/mohamad/Documents/GitHub/BrainQuest/data/stopword/words.txt"
stopwords = load_stopwords(stopwords_file_path)

In [ ]:
def tokenization_and_stemming(text):
    tokens = per_tokenize(text)
    filtered_tokens = remove_stopwords(tokens)
    stems = [stemmer.stem(token) for token in filtered_tokens]
    return " ".join(stems)

def per_tokenize(text):
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = text.lower()
    tokens = text.split()
    return tokens


def remove_stopwords(tokens):
    stop_words = set(stopwords)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

Read Documents


In [ ]:
import os
def parse_text(directory=DATA_URL):
    documents = []
    for filename in os.listdir(directory):
        if not filename.startswith("cran.all") or filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), "r") as file:
            text = file.read()
            documents = text.split(".I")
    return documents
documents = parse_text()
documents.remove(documents[0])

In [ ]:
stem_data = []
for document in documents:
    stem_data.append(tokenization_and_stemming(document))

In [ ]:
def create_word_count_list():
    word_count_list = []
    for document in stem_data:
        word_count = {}
        tokens = document.split()
        for token in tokens:
            token = token.strip('.,?!";:')
            if token in word_count:
                word_count[token] += 1
            else:
                word_count[token] = 1
        if word_count:
            del word_count[next(iter(word_count))]
        word_count_list.append(word_count)
    return word_count_list

word_count_list = create_word_count_list()


In [ ]:
def print_dic(word_count_list):
    for i, word_count in enumerate(word_count_list, start=1):
        print(f"\nDocument {i}:")
        for word, count in word_count.items():
            print(f"{word}: {count}")
        break

In [ ]:
def unique_words(word_count_list):
    unique_words = []
    for word_count in word_count_list:
        unique_words.extend(word_count)
    return set(unique_words)


In [ ]:
import openpyxl

# def write_to_excel(word_count_list, file_name):
#     wb = openpyxl.Workbook()
#     sheet = wb.active
#     sheet.title = "Word Count Data"

#     # Extracting all unique words
#     all_words = set()
#     for word_count in word_count_list:
#         all_words.update(word_count.keys())
#     all_words = sorted(all_words)

#     # Writing header row (document names)
#     sheet.cell(row=1, column=1).value = "Words"
#     for col, document in enumerate(range(2, len(word_count_list) + 2), start=2):
#         sheet.cell(row=1, column=col).value = f"Document {document - 1}"

#     # Writing data rows (word counts) in batches
#     batch_size = 10000  # Adjust batch size based on system memory
#     num_batches = (len(all_words) + batch_size - 1) // batch_size
#     for batch_index in range(num_batches):
#         start_index = batch_index * batch_size
#         end_index = min(start_index + batch_size, len(all_words))
#         for row, word in enumerate(all_words[start_index:end_index], start=2):
#             sheet.cell(row=row, column=1).value = word
#             for col, word_count in enumerate(word_count_list, start=2):
#                 count = word_count.get(word, 0)  # If word not found, default to 0
#                 sheet.cell(row=row, column=col).value = count

#     wb.save(file_name)


In [ ]:

word_count_list = create_word_count_list()
unique_words_list = unique_words(word_count_list)
len(unique_words_list)
print_dic(word_count_list)


In [ ]:
print_dic(word_count_list)
# write_to_excel(word_count_list, "word_count_data.xlsx")
word_count_list[0]
print("Number of unique words in all documents:", len(unique_words_list))

In [ ]:
# import pandas as pd
# df = pd.read_excel('/Users/mohamad/Documents/GitHub/BrainQuest/src/word_count_data.xlsx')
# print(df['Words'].head(100))